In [1]:
%pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
!pip install openmim
!mim install mmengine
!mim install mmcv==2.1.0
!pip install addict yapf timm torchvision

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 26.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.1+cu121
    Uninstalling torchaudio-2.5.1+cu121:
      Successfully uninstalled torchaudio-2.5.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 20.1 MB/s eta 0:0

In [2]:
import pandas as pd
import ast
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

In [3]:
!nvcc --version  # Check CUDA version
!pip show torch  # Check PyTorch version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Name: torch
Version: 2.1.0+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision


In [4]:
import mmcv
import mmengine
print("mmcv version:", mmcv.__version__)
print("mmengine version:", mmengine.__version__)

ModuleNotFoundError: No module named 'mmcv'

In [ ]:
print("Is GPU available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))

In [ ]:
%cd /content/drive/.shortcut-targets-by-id/1VitJZY6Stc4_5ydU3tFM8RZsBeoCFokO/nutri_estimate/depth_est_model

%pwd
!ls -l /content/drive/.shortcut-targets-by-id/1VitJZY6Stc4_5ydU3tFM8RZsBeoCFokO/nutri_estimate


In [ ]:
# Load the JSON files
classes = pd.read_json('../preprocess/classes.json')
df = pd.read_json('../preprocess/preprocessed_data.json', dtype={'label': 'str', 'total_mass': 'float64'})

In [ ]:
df = pd.read_json(
    '../preprocess/preprocessed_data.json',
    dtype={'label': 'str', 'total_mass': 'float64'}
)
df['label'] = df['label'].apply(ast.literal_eval)

In [ ]:
df.head(3)

In [ ]:
x_col = 'image_link'
y_col = 'label'
image_size = 320
input_shape = (image_size, image_size, 3)
directory = '../preprocess'

In [ ]:
X_train = df[df['split'] == 'train']
X_test = df[df['split'] == 'test']
X_train.shape, X_test.shape

In [ ]:
random_image_path = "../preprocess/" + X_test.sample(1)['image_link'].values[0]

In [ ]:
from PIL import Image
from torchvision import transforms

# Load image
image_path = random_image_path
image = Image.open(image_path).convert('RGB')

# Define preprocessing
preprocess = transforms.Compose([
    transforms.ToTensor(),  # Converts [H, W, C] -> [C, H, W]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization
])

# Preprocess and add batch dimension
input_tensor = preprocess(image).unsqueeze(0)  # Shape: [1, 3, H, W]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

In [ ]:
input_dict = {'input': input_tensor.to(device)}

In [ ]:
import torch
model = torch.hub.load('yvanyin/metric3d', 'metric3d_vit_small', pretrain=True)
model = model.to(device)
pred_depth, confidence, output_dict = model.inference(input_dict)
pred_normal = output_dict['prediction_normal'][:, :3, :, :] # only available for Metric3Dv2 i.e., ViT models
normal_confidence = output_dict['prediction_normal'][:, 3, :, :] # see https://arxiv.org/abs/2109.09881 for details


intrinsic = [707.0493, 707.0493, 604.0814, 180.5066]
canonical_to_real_scale = intrinsic[0] / 1000.0 # 1000.0 is the focal length of canonical camera
pred_depth = pred_depth * canonical_to_real_scale # now the depth is metric
pred_depth = torch.clamp(pred_depth, 0, 300)

In [ ]:
import matplotlib.pyplot as plt

# Display the image
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.axis('off')
plt.title(f"Actual image", fontsize=12)
plt.show()

# Convert depth to numpy for visualization
pred_depth_np = pred_depth.squeeze().detach().cpu().numpy()

# Plot depth map
plt.imshow(pred_depth_np, cmap='plasma')
plt.colorbar()
plt.title('Predicted Depth')
plt.show()

In [ ]:
import numpy as np

# Convert depth tensor to a NumPy array
depth_values = pred_depth.squeeze().cpu().numpy()  # Shape: [H, W]

# Display specific value from centre (i am assuming the object is in the centre of the image)
print("Depth at (150, 150):", depth_values[150, 150], "meters")

# # Save the depth map as a file for calculation(?)
# np.save("absolute_depth_map.npy", depth_values)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [1]:
!pip install openmim
!mim install mmengine
!mim install mmcv==2.1.0
!pwd

Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.1.0/index.html
  Using cached mmengine-0.10.5-py3-none-any.whl.metadata (20 kB)
Using cached mmengine-0.10.5-py3-none-any.whl (452 kB)
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.1.0/index.html
  Using cached https://download.openmmlab.com/mmcv/dist/cu121/torch2.1.0/mmcv-2.1.0-cp310-cp310-manylinux1_x86_64.whl (94.1 MB)
/content


In [1]:
%cd /content/drive/MyDrive/Colab_Notebooks/computer_vision_app
!pwd
!ls -l
!pip install -r requirements.txt

/content/drive/MyDrive/Colab_Notebooks/computer_vision_app
/content/drive/MyDrive/Colab_Notebooks/computer_vision_app
total 14588
-rw------- 1 root root     4763 Dec  8 05:49 app.py
-rw------- 1 root root      359 Dec  8 01:22 _data_base_.py
drwx------ 2 root root     4096 Dec  8 04:34 images
drwx------ 2 root root     4096 Dec  8 04:34 models
-rw------- 1 root root 14918622 Dec  8 05:11 ngrok
-rw------- 1 root root      176 Dec  8 04:34 README.md.gdoc
-rw------- 1 root root      887 Dec  8 06:02 requirements.txt
drwx------ 2 root root     4096 Dec  8 04:34 utilities


In [2]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 1s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [3]:
!curl https://loca.lt/mytunnelpassword
!streamlit run app.py & lt --port 8501

34.142.225.33

your url is: https://slick-steaks-agree.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.142.225.33:8501

2024-12-08 06:05:34.974 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2024-12-08 06:06:17.663 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: 